In [1]:
import torch

In [2]:
if torch.backends.mps.is_available():
    mps_device = torch.device("mps")
    x = torch.ones(1, device=mps_device)
    print (x)
else:
    print ("MPS device not found.")

tensor([1.], device='mps:0')


In [5]:
import sys
import platform
import torch
import pandas as pd
import sklearn as sk

has_gpu = torch.cuda.is_available()
has_mps = torch.backends.mps.is_available()
device = "mps" if has_mps \
    else "gpu" if has_gpu else "cpu"

print(f"Python Platform: {platform.platform()}")
print(f"PyTorch Version: {torch.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
print("GPU is", "available" if has_gpu else "NOT AVAILABLE")
print("MPS (Apple Metal) is", "AVAILABLE" if has_mps else "NOT AVAILABLE")
print(f"Target device is {device}")

Python Platform: macOS-13.4-arm64-arm-64bit
PyTorch Version: 2.1.0.dev20230618

Python 3.11.4 (main, Jun  7 2023, 00:34:59) [Clang 14.0.3 (clang-1403.0.22.14.1)]
Pandas 2.0.2
Scikit-Learn 1.2.2
GPU is NOT AVAILABLE
MPS (Apple Metal) is AVAILABLE
Target device is mps


### MNIST IN PYTORCH ON APPLE SILICON GPU

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
    
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx*len(data)}/{len(train_loader.dataset)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction="sum").item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)
    print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)\n")


def main():
    print(f"Target device is {device}")
    print("Loading MNIST dataset...")
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST(
            root=".", train=True, download=True,
            transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))
            ])
        ),
        batch_size=64, shuffle=True
    )
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(
            root=".", train=False, download=True,
            transform=transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))
            ])
        ),
        batch_size=1000, shuffle=True
    )
    print("Done.")

    model = Net().to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

    for epoch in range(1, EPOCHS + 1):
        train(model, device, train_loader, optimizer, epoch)
        test(model, device, test_loader)

if __name__ == "__main__":
    EPOCHS = 5
    main()

5.0%

Target device is mps
Loading MNIST dataset...


100.0%


Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw



100.0%
65.6%

Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw



100.0%
100.0%


Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw

Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw

Done.
Train Epoch: 1 [0/60000 (0%)]	Loss: 2.310721
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.606480
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.479934
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.247395
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.186194
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.222960
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.103005
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.099257
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.049124
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.082247

Test set: Average loss: 0.0898, Accuracy: 9735/10000 (97%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.144750
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.121607
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.088173
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.022042
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.049248
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.064193
Train Epoch: 2 